In [3]:
import sys,os
sys.path.append('../api')
import dms3

class DbAttrProxy(object):
    '''Proxy attribute access for database entries, for use with expression evaluation.'''
    def __init__(self,DB,klass,id):
        self.DB,self.klass,self.id=DB,klass,id
        self.schema,self.vals=dms3.GG.db_get_schema_object(self.DB,self.klass,self.id)
    def __getattr__(self,attr):
        if attr not in self.schema: raise AttributeError(f'{self.klass}.{attr}: no such attribute.')
        item=self.schema[attr]
        if attr not in val: raise ValueError(f'{self.klass}.{attr}: exists in schema but absent from database (id={self.id}).')
        val=self.vals[attr]
        if item.link is None:
            # TODO: preprocess values?
            # e.g. wrap dicts as attrdict.AttrDict, or convert to astropy.Quantity etc.
            return val
        else:
            # handle links
            scalar=(len(item.shape)==0)
            if scalar: return DbAttrProxy(DB=self.DB,klass=item.link,id=val)
            else: return [DbAttrProxy(DB=self.DB,klass=item.link,id=v) for v in val]
    def __dir__(self):
        return self.schema.keys()
    def __repr__(self):
        return f'<{self.klass} proxy: DB={self.DB} dbId={self.id}>'
    def _self_dict(self):
        'Returns self dictionary, for populating top-level namespace'
        return dict([(k,self.__getattr__(k)) for k in self.schema.keys()])

DB='dms0'
REST_URL='http://localhost:8080'
ID0=dms3.dms_api_object_list(db=DB,type='BeamState')[0]
p=DbAttrProxy(DB=DB,klass='BeamState',id=ID0)

In [4]:
import asteval
from rich.pretty import pprint
aeval=asteval.Interpreter()
import pydantic
import typing
aeval.symtable|=p._self_dict()
aeval('beam.height["value"]')

UnboundLocalError: local variable 'val' referenced before assignment

In [39]:
aeval('all([i<5 for i in range(10)])').__class__

numpy.bool_